In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

In [2]:
imdb=tfds.load('imdb_reviews')

In [3]:
train_data,test_data=imdb['train'],imdb['test']

In [17]:
train_sen,train_label=[],[]
for lines in train_data:
    train_sen.append(str(lines['text'].numpy()))
    train_label.append(lines['label'].numpy())

In [18]:
train_sen[2]

"b'Mann photographs the Alberta Rocky Mountains in a superb fashion, and Jimmy Stewart and Walter Brennan give enjoyable performances as they always seem to do. <br /><br />But come on Hollywood - a Mountie telling the people of Dawson City, Yukon to elect themselves a marshal (yes a marshal!) and to enforce the law themselves, then gunfighters battling it out on the streets for control of the town? <br /><br />Nothing even remotely resembling that happened on the Canadian side of the border during the Klondike gold rush. Mr. Mann and company appear to have mistaken Dawson City for Deadwood, the Canadian North for the American Wild West.<br /><br />Canadian viewers be prepared for a Reefer Madness type of enjoyable howl with this ludicrous plot, or, to shake your head in disgust.'"

In [19]:
test_sen,test_label=[],[]
for lines in test_data:
    test_sen.append(str(lines['text'].numpy()))
    test_label.append(lines['label'].numpy())

In [20]:
train_sen=np.asarray(train_sen)
test_sen=np.asarray(test_sen)
train_label=np.asarray(train_label,dtype=np.int32)
test_label=np.asarray(test_label,dtype=np.int32)

In [21]:
train_sen.shape

(25000,)

In [22]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [24]:
token=Tokenizer(num_words=10000,oov_token='<OOV>')
token.fit_on_texts(train_sen)
word_index=token.word_index
vocab_size=len(word_index)
print(vocab_size)

86539


In [25]:
train_seq=token.texts_to_sequences(train_sen)
test_seq=token.texts_to_sequences(test_sen)

In [26]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [27]:
train_pad_seq=pad_sequences(train_seq,maxlen=15,padding='post')
test_pad_seq=pad_sequences(test_seq,maxlen=15,padding='post')


In [28]:
from tensorflow.keras.layers import Embedding,GlobalAveragePooling1D,Dense

In [29]:
model=keras.Sequential()
model.add(Embedding(input_dim=10000,output_dim=32,input_length=15))
model.add(GlobalAveragePooling1D())
model.add(Dense(units=128,activation='relu'))
model.add(Dense(units=1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 15, 32)            320000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 128)               4224      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 324,353
Trainable params: 324,353
Non-trainable params: 0
_________________________________________________________________


In [33]:
model.fit(train_pad_seq,train_label,validation_data=(test_pad_seq,test_label),epochs=10)

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 10s 409us/sample - loss: 0.5604 - accuracy: 0.7050 - val_loss: 0.5096 - val_accuracy: 0.7434
Epoch 2/10
25000/25000 [==============================] - 10s 395us/sample - loss: 0.4452 - accuracy: 0.7921 - val_loss: 0.5120 - val_accuracy: 0.7441
Epoch 3/10
25000/25000 [==============================] - 10s 406us/sample - loss: 0.4004 - accuracy: 0.8185 - val_loss: 0.5258 - val_accuracy: 0.7408
Epoch 4/10
25000/25000 [==============================] - 9s 367us/sample - loss: 0.3663 - accuracy: 0.8340 - val_loss: 0.5550 - val_accuracy: 0.7310
Epoch 5/10
25000/25000 [==============================] - 10s 386us/sample - loss: 0.3286 - accuracy: 0.8523 - val_loss: 0.6030 - val_accuracy: 0.7258
Epoch 6/10
25000/25000 [==============================] - 10s 382us/sample - loss: 0.2933 - accuracy: 0.8657 - val_loss: 0.6769 - val_accuracy: 0.7218
Epoch 7/10
25000/25000 [=====================